In [18]:
from datetime import datetime

from gcsa.google_calendar import GoogleCalendar

calendar = GoogleCalendar("kofsitho@gmail.com")
for event in calendar.get_events(order_by="startTime", single_events=True):
    print(event)

    # calendar.delete_event(event)

2026-09-04 - Happy birthday!


In [32]:
from datetime import timedelta

time_min = datetime.strptime("2025-10-31 00:00:00", "%Y-%m-%d %H:%M:%S").date()

print(time_min)

for event in calendar.get_events(
    time_min=time_min, time_max=datetime.now() + timedelta(days=1)
):
    print(event)

2025-10-31
2025-10-31 - Agent 워크샵


In [25]:
for event in calendar.get_events(
    time_min=datetime.now() - timedelta(days=30),
    time_max=datetime.now() - timedelta(days=1),
):
    print(event)
    calendar.delete_event(event)

2025-10-02 09:00:00+09:00 - ✅ 📧 Morning Catch Up
2025-10-02 09:30:00+09:00 - ✅ 🏃 Sprint Planning
2025-10-02 12:00:00+09:00 - ✅ 🍱 Lunch
2025-10-03 12:00:00+09:00 - ✅ 🍱 Lunch
2025-10-03 09:00:00+09:00 - ✅ 📧 Morning Catch Up
2025-10-06 09:00:00+09:00 - ✅ 📧 Morning Catch Up
2025-10-06 12:00:00+09:00 - ✅ 🍱 Lunch
2025-10-07 09:00:00+09:00 - ✅ 📧 Morning Catch Up
2025-10-07 12:00:00+09:00 - ✅ 🍱 Lunch
2025-10-08 09:00:00+09:00 - ✅ 📧 Morning Catch Up
2025-10-08 12:00:00+09:00 - ✅ 🍱 Lunch
2025-10-09 09:00:00+09:00 - ✅ 📧 Morning Catch Up
2025-10-09 12:00:00+09:00 - ✅ 🍱 Lunch
2025-10-10 12:00:00+09:00 - ✅ 🍱 Lunch
2025-10-10 09:00:00+09:00 - ✅ 📧 Morning Catch Up
2025-10-10 09:30:00+09:00 - ✅ 🏃 Sprint Planning
2025-10-13 09:00:00+09:00 - ✅ 📧 Morning Catch Up
2025-10-13 12:00:00+09:00 - ✅ 🍱 Lunch
2025-10-14 12:00:00+09:00 - ✅ 🍱 Lunch
2025-10-14 09:00:00+09:00 - ✅ 📧 Morning Catch Up
2025-10-15 09:00:00+09:00 - ✅ 📧 Morning Catch Up
2025-10-15 12:00:00+09:00 - ✅ 🍱 Lunch
2025-10-17 09:00:00+09:00 - ✅ 📧 Mo

In [21]:
from datetime import datetime

from gcsa.event import Event

date = datetime.strptime("2025-10-31 15:00:00", "%Y-%m-%d %H:%M:%S").date()

event = Event(
    "Agent 워크샵",
    start=date,
    location="경기도 판교 와이즈에이아이",
    minutes_before_popup_reminder=15,
)
calendar.add_event(event)

<Event 2025-10-31 - Agent 워크샵>

In [36]:
import base64
from email.mime.text import MIMEText

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = [
    "https://www.googleapis.com/auth/gmail.send",
    "https://www.googleapis.com/auth/gmail.readonly",
]
creds = None
flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
creds = flow.run_local_server(port=0)
service = build("gmail", "v1", credentials=creds)

message = MIMEText("이메일 본문")
message["to"] = "kofsitho@naver.com"
message["subject"] = "안녕 지메일 보내기 테스트"
raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
body = {"raw": raw}

sent = service.users().messages().send(userId="me", body=body).execute()
print("Sent message id:", sent["id"])


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=418180406197-nnv08m694ba5shah443cm04l7mjsj3is.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64294%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=Ho05zEwyAiFbKmCIssEIsXelUSNZsO&access_type=offline
Sent message id: 19a382650e5c18a2


In [39]:
import base64

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Gmail 읽기/전송용 스코프
SCOPES = [
    "https://www.googleapis.com/auth/gmail.readonly",
    "https://www.googleapis.com/auth/gmail.send",
]


def get_service():
    flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
    creds = flow.run_local_server(port=0)
    return build("gmail", "v1", credentials=creds)


service = get_service()

# 최근 이메일 10개 가져오기
results = service.users().messages().list(userId="me", maxResults=10).execute()
messages = results.get("messages", [])

for msg in messages:
    msg_id = msg["id"]
    msg_data = (
        service.users().messages().get(userId="me", id=msg_id, format="full").execute()
    )

    headers = msg_data["payload"]["headers"]
    subject = next(
        (h["value"] for h in headers if h["name"] == "Subject"), "(제목 없음)"
    )
    sender = next((h["value"] for h in headers if h["name"] == "From"), "(보낸이 없음)")

    # 본문 가져오기
    if "parts" in msg_data["payload"]:
        data = msg_data["payload"]["parts"][0]["body"].get("data")
    else:
        data = msg_data["payload"]["body"].get("data")

    if data:
        decoded = base64.urlsafe_b64decode(data)
        msg_body = decoded.decode("utf-8", errors="ignore")
    else:
        msg_body = "(본문 없음)"

    print(f"📩 제목: {subject}")
    print(f"보낸 사람: {sender}")
    print(f"내용 요약: {msg_body[:100]}...\n")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=418180406197-nnv08m694ba5shah443cm04l7mjsj3is.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64883%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=SNgzIJiGWO3xktNxHB3jkw6jEi4TGl&access_type=offline
📩 제목: Re: [wiseai-oneplat/wiseai-aiu] <AIUDEV-1969> 수동 아웃바운드 적재시 필터 조건 변경 (PR #1482)
보낸 사람: danielle0721 <notifications@github.com>
내용 요약: Merged #1482 into qa.

-- 
Reply to this email directly or view it on GitHub:
https://github.com...

📩 제목: [wiseai-oneplat/wiseai-aiu] <AIUDEV-1969> 수동 아웃바운드 적재시 필터 조건 변경 (PR #1482)
보낸 사람: danielle0721 <notifications@github.com>
내용 요약: ## :bulb: 관련 정보

- JIRA 티켓: [AIUDEV-1969](https://wisecareai.atlassian.net/browse/AIUDEV-1969)

...

📩 제목: (제목 없음)
보낸 사람: kofsitho@gmail.com
내용 요약: 이메일 본문...

📩 제목: 보안 알림
보낸 사람: Google <no-reply@accounts.goog

In [ ]:
import yagmail

yag = yagmail.SMTP("kofsitho@gmail.com", oauth2_file="credentials.json")
yag.send(
    to="kofsitho@naver.com",
    subject="안녕하세요! shw님",
    contents="오늘 오전 10시에 미팅이 있습니다.",
)